### Model Credit Risk in Loans

### Introduction

####  Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

#### Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data and their own data science process and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back.

#### A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

#### Investors are primarily interested in receiveing a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

#### The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.


### Data

#### Lending Club releases data for all of the approved and declined loan applications periodically on their website. 

#### Before diving into the datasets themselves, let's get familiar with the data dictionary. The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

#### The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

   - Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

#### Before we can start doing machine learning, we need to define what features we want to use and which column repesents the target column we want to predict.

### Explore the data 

In [5]:
# Read loans_2007_2011.csv into a DataFrame named loans_2007
import pandas as pd
loans_2007 = pd.read_csv("loans_2007_2011.csv", low_memory=False)
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.info())
loans_2007.head()

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


#### The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. Pay attention to any features that:

   - leak information from the future (after the loan has already been funded)
   - don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
   - formatted poorly and need to be cleaned up
   - require more data or a lot of processing to turn into a useful feature
   - contain redundant information

#### We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans.

### Data Cleaning

#### After analyzing each column, we can conclude that the following features need to be removed:

   - id: randomly generated field by Lending Club for unique identification purposes only
   - member_id: also a randomly generated field by Lending Club for unique identification purposes only
   - funded_amnt: leaks data from the future (after the loan is already started to be funded)
   - funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
   - grade: contains redundant information as the interest rate column (int_rate)
   - sub_grade: also contains redundant information as the interest rate column (int_rate)
   - emp_title: requires other data and a lot of processing to potentially be useful
   - issue_d: leaks data from the future (after the loan is already completed funded)

#### Recall that Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

#### Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

   - id
   - member_id
   - funded_amnt
   - funded_amnt_inv
   - grade
   - sub_grade
   - emp_title
   - issue_d

In [6]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

#### Next, we need to drop the following columns:

   - zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
   - out_prncp: leaks data from the future, (after the loan already started to be paid off)
   - out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
   - total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
   - total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
   - total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)

#### The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

#### Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

  -  zip_code
  -  out_prncp
  -  out_prncp_inv
  -  total_pymnt
  -  total_pymnt_inv
  -  total_rec_prncp

In [7]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

#### Next, we need to drop the following columns:

   - total_rec_int: leaks data from the future, (after the loan already started to be paid off),
   - total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
   - recoveries: also leaks data from the future, (after the loan already started to be paid off),
   - collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
   - last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
   - last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

#### All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

#### Use the Dataframe method drop to remove the following columns from the loans_2007 Dataframe:

   - total_rec_int
   - total_rec_late_fee
   - recoveries
   - collection_recovery_fee
   - last_pymnt_d
   - last_pymnt_amnt

In [8]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.info())

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

#### Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

#### We should use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. 

#### Use the Series method value_counts to return the frequency of the unique values in the loan_status column.

In [9]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

#### There are 8 different possible values for the loan_status column. 
#### The each column explanation for each column:

- Fully Paid: Loan has been fully paid off.
- Charged Off: Loan for which there is no longer a reasonable expectation of further payments.
- Does not meet the credit policy. Status:Fully Paid: While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- Does not meet the credit policy. Status:Charged Off: While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- In Grace Period: The loan is past due but still in the grace period of 15 days.
- Late (16-30 days): Loan hasn't been paid in 16 to 30 days (late on the current payment).
- Late (31-120 days): Loan hasn't been paid in 31 to 120 days (late on the current payment).
- Current: Loan is up to date on current payments.
- Default: Loan is defaulted on and no payment has been made for more than 121 days.

#### From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance.

#### Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case.

#### Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. 

#### Remove all rows from loans_2007 that contain values other than Fully Paid or Charged Off for the loan_status column. Use the Dataframe method replace to replace categorical data to numerical.

In [10]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

#### let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application. 

#### Since we're trying to find columns that contain one true unique value, we should first drop the null values then compute the number of unique values.

In [11]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


#### It looks we we were able to remove 9 more columns since they only contained 1 unique value.

### Feature Engineering

#### We removed all of the columns that contained redundant information, weren't useful for modeling, required too much processing to make useful, or leaked information from the future. 

####  We'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process. 

#### This is because the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, scikit-learn will return an error if you try to train a model using data that contain missing values or non-numeric values when working with models like linear regression and logistic regression.

#### We can return the number of missing values across the Dataframe by:

   - first using the Pandas Dataframe method isnull to return a Dataframe containing Boolean values:
       - True if the original value is null,
       - False if the original value isn't null.
   - then using the Pandas Dataframe method sum to calculate the number of null values in each column.

In [12]:
loans = loans_2007
null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


#### While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

#### This means that we'll keep the following columns and just remove rows containing missing values for them:

   - title
   - revol_util
   - last_credit_pull_d

#### and drop the pub_rec_bankruptcies column entirely since more than 1% of the rows have a missing value for this column.

#### Next:

   - Use the drop method to remove the pub_rec_bankruptcies column from loans.
   - Use the dropna method to remove all rows from loans containing any missing values.
   - Use the dtypes attribute followed by the value_counts() method to return the counts for each column data type. Use the print function to display these counts.


In [13]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


#### While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. 
   - Use the Dataframe method select_dtypes to select only the columns of object type from loans and assign the resulting Dataframe object_columns_df.
   - Display the first row in object_columns_df using the print function.


In [14]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


#### Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

   - home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
   - verification_status: indicates if income was verified by Lending Club,
   - emp_length: number of years the borrower was employed upon time of application,
   - term: number of payments on the loan, either 36 or 60,
   - addr_state: borrower's state of residence,
   - purpose: a category provided by the borrower for the loan request,
   - title: loan title provided the borrower,

#### There are also some columns that represent numeric values, that need to be converted:

   - int_rate: interest rate of the loan in %,
   - revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more here.

#### Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

#### Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

   - earliest_cr_line: The month the borrower's earliest reported credit line was opened,
   - last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

#### Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

#### Display the unique value counts for the following columns: home_ownership, verification_status, emp_length, term, addr_state columns:

   - Store these column names in a list named cols.
   - Use a for loop to iterate over cols:
       - Use the print function combined with the Series method value_counts to display each column's unique value counts.

In [15]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18513
MORTGAGE    17112
OWN          2984
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16696
Verified           12290
Source Verified     9722
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1229
n/a          1032
Name: emp_length, dtype: int64
 36 months    29041
 60 months     9667
Name: term, dtype: int64
CA    6958
NY    3713
FL    2791
TX    2667
NJ    1798
IL    1483
PA    1473
VA    1376
GA    1364
MA    1301
OH    1179
MD    1026
AZ     850
WA     822
CO     770
NC     753
CT     730
MI     712
MO     671
MN     603
NV     481
SC     462
WI     441
AL     437
OR     436
LA     430
KY     315
OK     290
KS     260
UT     254
AR     237
DC     209
RI     196
NM     184
WV     172
HI     166
NH     166
DE     113
MT      83
WY      80
AK      

#### The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

#### First, let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

#### Use the value_counts method and the print function to display the unique values in the following columns:

   - purpose
   - title

In [16]:
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    18130
credit_card            5039
other                  3864
home_improvement       2897
major_purchase         2155
small_business         1762
car                    1510
wedding                 929
medical                 680
moving                  576
vacation                375
house                   369
educational             320
renewable_energy        102
Name: purpose, dtype: int64
Debt Consolidation                                      2104
Debt Consolidation Loan                                 1632
Personal Loan                                            642
Consolidation                                            494
debt consolidation                                       485
Credit Card Consolidation                                353
Home Improvement                                         346
Debt consolidation                                       324
Small Business Loan                                      310
Credit Card Loan               

#### The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

#### It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

#### We can use the following mapping to clean the emp_length column:

   - "10+ years": 10
   - "9 years": 9
   - "8 years": 8
   - "7 years": 7
   - "6 years": 6
   - "5 years": 5
   - "4 years": 4
   - "3 years": 3
   - "2 years": 2
   - "1 year": 1
   - "< 1 year": 0
   - "n/a": 0

#### We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

#### Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.


   - Remove the last_credit_pull_d, addr_state, title, and earliest_cr_line columns from loans.
   - Convert the int_rate and revol_util columns to float columns by:
       - Using the str acessor followed by the rstrip string method to strip the right trailing percent sign (%):
           - loans['int_rate'].str.rstrip('%') returns a new Series with % stripped from the right side of each value.
       - On the resulting Series object, use the astype method to convert to the float type.
       - Assign the new Series of float values back to the respective columns in the Dataframe.
   - Use the replace method to clean the emp_length column.

In [17]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

#### Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model. We first need to use the Pandas get_dummies method to return a new Dataframe containing a new column for each dummy variable. We can then use the concat method to add these dummy columns back to the original Dataframe. 

- Encode the home_ownership, verification_status, purpose, and term columns as integer values:

   - Use the get_dummies function to return a Dataframe containing the dummy columns.
   - Use the concat method to add these dummy columns back to loans.
   - Remove the original, non-dummy columns (home_ownership, verification_status, purpose, and term) from loans.

In [18]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

#### We performed data preparation necessary to start training machine learning models. We converted all of the columns to numerical values because those are the only type of value scikit-learn can work with. 

In [19]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38708 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              38708 non-null float64
int_rate                               38708 non-null float64
installment                            38708 non-null float64
emp_length                             38708 non-null int64
annual_inc                             38708 non-null float64
loan_status                            38708 non-null int64
dti                                    38708 non-null float64
delinq_2yrs                            38708 non-null float64
inq_last_6mths                         38708 non-null float64
open_acc                               38708 non-null float64
pub_rec                                38708 non-null float64
revol_bal                              38708 non-null float64
revol_util                             38708 non-null float64
total_acc                              38708 non-null float64
home_ownership_MORTGAGE    

### Selecting Error Metric

####  Recall the original question we wanted to answer:

   - Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

#### We established that this is a binary classification problem in the first mission of this course, and we converted the loan_status column to 0s and 1s as a result. Before diving in and selecting an algorithm to apply to the data, we should select an error metric.

#### An error metric will help us figure out when our model is performing well, and when it's performing poorly. To tie error metrics all the way back to the original question we wanted to answer, let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

#### In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off. 

#### In the loan_status and prediction columns, a 0 means that the loan wouldn't be paid off on time, and a 1 means that it would.

#### Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).


   - Find the number of true negatives.
       - Find the number of items where predictions is 0, and the corresponding entry in loans["loan_status"] is also 0.
       - Assign the result to tn.
   - Find the number of true positives.
       - Find the number of items where predictions is 1, and the corresponding entry in loans["loan_status"] is also 1.
       - Assign the result to tp.
   - Find the number of false negatives.
       - Find the number of items where predictions is 0, and the corresponding entry in loans["loan_status"] is 1.
       - Assign the result to fn.
   - Find the number of false positives.
       - Find the number of items where predictions is 1, and the corresponding entry in loans["loan_status"] is 0.
       - Assign the result to fp.

In [23]:
import pandas as pd
import numpy as np
predictions = pd.Series(np.ones(loans.shape[0]))
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

#### We mentioned earlier that there is a significant class imbalance in the loan_status column. There are 6 times as many loans that were paid off on time (1), than loans that weren't paid off on time (0). This causes a major issue when we use accuracy as a metric. This is because due to the class imbalance, a classifier can predict 1 for every row, and still have high accuracy. 

#### This is why it's important to always be aware of imbalanced classes in machine learning models, and to adjust your error metric accordingly. In this case, we don't want to use accuracy, and should instead use metrics that tell us the number of false positives and false negatives.

#### This means that we should optimize for:

   - high recall (true positive rate)
   - low fall-out (false positive rate)

#### We can calculate false positive rate and true positive rate, using the numbers of true positives, true negatives, false negatives, and false positives.

- fpr = fp / (fp + tn)

#### False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off.
- tpr = tp / (tp + fn)

#### Simple english ways to think of each term are:

   - False Positive Rate -- "what percentage of my 1 predictions are incorrect?"
       - In this case, "what percentage of the loans that I fund would not be repaid?"
   - True Positive Rate -- "what percentage of all the possible 1 predictions am I making?"
       - In this case, "what percentage of loans that could be funded would I fund?"

#### Generally, if we want to reduce false positive rate, true positive rate will also go down. This is because if we want to reduce the risk of false positives, we wouldn't think about funding riskier loans in the first place.

- Compute the false positive rate for predictions.
- Compute the true positive rate for predictions.

In [24]:
# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

1.0
1.0


#### Notice that both fpr and tpr were 1. This is because we predicted 1 for each row. This means that we correctly identified all of the good loans (true positive rate), but we also incorrectly identified all of the bad loans (false positive rate). Now that we've setup error metrics, we can move on to making predictions using a machine learning algorithm.

### Model Training 

#### A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:

   - it's quick to train and we can iterate more quickly,
   - it's less prone to overfitting than more complex models like decision trees,
   - it's easy to interpret.
   
### Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)

#### In order to get a realistic depiction of the accuracy of the model, let's perform k-fold cross validation. We can use the cross_val_predict() function from the sklearn.model_selection package.

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.9987266289831667
0.9977929004965974


#### Our fpr and tpr are around what we'd expect if the model was predicting all ones. 

#### Unfortunately, even through we're not using accuracy as an error metric, the classifier is, and it isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes. The two main ways are:

   - Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
   - Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

#### We'll look into oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where loan_status is 0, and where loan_status is 1. This way, the classifier is forced to make actual predictions, since predicting all 1s or all 0s will only result in 50% accuracy at most.

#### The downside of this technique is that since it has to preserve an equal ratio, you have to either:

   - Throw out many rows of data. If we wanted equal numbers of rows where loan_status is 0 and where loan_status is 1, one way we could do that is to delete rows where loan_status is 1.
   - Copy rows multiple times. One way to equalize the 0s and 1s is to copy rows where loan_status is 0.
   - Generate fake data. One way to equalize the 0s and 1s is to generate new rows where loan_status is 0.

#### Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn.

#### We can do this by setting the class_weight parameter to balanced when creating the LogisticRegression instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

#### By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. You can read more about the parameter here. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

#### We can repeat the cross validation procedure we performed in the last screen, but with the class_weight parameter set to balanced.


   - Create a LogisticRegression instance.
       - set class_weight to balanced.
       - Assign the instance to lr.
   - Generate cross validated predictions for features.
       - Call cross_val_predict() using lr, features, and target.
       - Assign the predictions to predictions.
   - Use the Series class to convert predictions to a Pandas Series, as we did in the last screen.
       - Converting to Series objects letse us take advantage of boolean filtering and arithmetic operations from pandas.
   - Compute true positive rate and false positive rate.
       - Assign true positive rate to tpr.
       - Assign false positive rate to fpr.
   - Print out fpr and tpr to evaluate them.


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.618423504565501
0.6220342100423027


#### We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. Our true positive rate is now around 61%, and our false positive rate is around 62%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund 61% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

#### We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty.

#### We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the class_weight parameter:

penalty = {

    0: 10,

    1: 1

}

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.21718740294428227
0.22696339893323525


#### It looks like assigning manual penalties lowered the false positive rate to 22%, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

#### While we could tweak the penalties further, it's best to move to trying a different model right now, for larger potential false positive rate gains. We can always loop back and iterate on the penalties more later.

### Random Forest

#### Let's try a more complex algorithm, random forest. Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.

#### We can use the RandomForestClassifer class from scikit-learn to do this.

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9648425368035282
0.966525657531727


#### Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

#### Ultimately, our best model had a false positive rate of 7%, and a true positive rate of 20%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 7% of borrowers defaulting, and that the pool of 20% of borrowers is large enough to make enough interest money to offset the losses.

#### If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

   - We can tweak the penalties further.
   - We can try models other than a random forest and logistic regression.
   - We can use some of the columns we discarded to generate better features.
   - We can ensemble multiple models to get more accurate predictions.
   - We can tune the parameters of the algorithm to achieve higher performance.
